In [1]:
import pandas as pd
import itertools
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
tqdm.pandas()

# Summary

Face recognition models are regular convolutional neural networks models. They represent face photos as vectors. We find the distance between these two vectors to compare two faces. Finally, we classify two faces as same person whose distance is less than a threshold value.

The question is that how to determine the threshold. In this notebook, we will find the best split point for a threshold.

# Data set

In [2]:
# Ref: https://github.com/serengil/deepface/tree/master/tests/dataset
idendities = {
    "Angelina": ["img1.jpg", "img2.jpg", "img4.jpg", "img5.jpg", "img6.jpg", "img7.jpg", "img10.jpg", "img11.jpg"],
    "Scarlett": ["img8.jpg", "img9.jpg"],
    "Jennifer": ["img3.jpg", "img12.jpg"],
    "Mark": ["img13.jpg", "img14.jpg", "img15.jpg"],
    "Jack": ["img16.jpg", "img17.jpg"],
    "Elon": ["img18.jpg", "img19.jpg"],
    "Jeff": ["img20.jpg", "img21.jpg"],
    "Marissa": ["img22.jpg", "img23.jpg"],
    "Sundar": ["img24.jpg", "img25.jpg"]
}

# Positive samples
Find different photos of same people

In [3]:
positives = []

for key, values in idendities.items():
    
    print(key)
    print(values)
    for i in range(0, len(values)-1):
        for j in range(i+1, len(values)):
            #print(values[i], " and ", values[j])
            positive = []
            positive.append(values[i])
            positive.append(values[j])
            positives.append(positive)

print(positives)

Angelina
['img1.jpg', 'img2.jpg', 'img4.jpg', 'img5.jpg', 'img6.jpg', 'img7.jpg', 'img10.jpg', 'img11.jpg']
Scarlett
['img8.jpg', 'img9.jpg']
Jennifer
['img3.jpg', 'img12.jpg']
Mark
['img13.jpg', 'img14.jpg', 'img15.jpg']
Jack
['img16.jpg', 'img17.jpg']
Elon
['img18.jpg', 'img19.jpg']
Jeff
['img20.jpg', 'img21.jpg']
Marissa
['img22.jpg', 'img23.jpg']
Sundar
['img24.jpg', 'img25.jpg']
[['img1.jpg', 'img2.jpg'], ['img1.jpg', 'img4.jpg'], ['img1.jpg', 'img5.jpg'], ['img1.jpg', 'img6.jpg'], ['img1.jpg', 'img7.jpg'], ['img1.jpg', 'img10.jpg'], ['img1.jpg', 'img11.jpg'], ['img2.jpg', 'img4.jpg'], ['img2.jpg', 'img5.jpg'], ['img2.jpg', 'img6.jpg'], ['img2.jpg', 'img7.jpg'], ['img2.jpg', 'img10.jpg'], ['img2.jpg', 'img11.jpg'], ['img4.jpg', 'img5.jpg'], ['img4.jpg', 'img6.jpg'], ['img4.jpg', 'img7.jpg'], ['img4.jpg', 'img10.jpg'], ['img4.jpg', 'img11.jpg'], ['img5.jpg', 'img6.jpg'], ['img5.jpg', 'img7.jpg'], ['img5.jpg', 'img10.jpg'], ['img5.jpg', 'img11.jpg'], ['img6.jpg', 'img7.jpg'], ['img6

In [4]:
positives = pd.DataFrame(positives, columns = ["file_x", "file_y"])
positives["decision"] = "Yes"
positives

file_x     file_y decision
0    img1.jpg   img2.jpg      Yes
1    img1.jpg   img4.jpg      Yes
2    img1.jpg   img5.jpg      Yes
3    img1.jpg   img6.jpg      Yes
4    img1.jpg   img7.jpg      Yes
5    img1.jpg  img10.jpg      Yes
6    img1.jpg  img11.jpg      Yes
7    img2.jpg   img4.jpg      Yes
8    img2.jpg   img5.jpg      Yes
9    img2.jpg   img6.jpg      Yes
10   img2.jpg   img7.jpg      Yes
11   img2.jpg  img10.jpg      Yes
12   img2.jpg  img11.jpg      Yes
13   img4.jpg   img5.jpg      Yes
14   img4.jpg   img6.jpg      Yes
15   img4.jpg   img7.jpg      Yes
16   img4.jpg  img10.jpg      Yes
17   img4.jpg  img11.jpg      Yes
18   img5.jpg   img6.jpg      Yes
19   img5.jpg   img7.jpg      Yes
20   img5.jpg  img10.jpg      Yes
21   img5.jpg  img11.jpg      Yes
22   img6.jpg   img7.jpg      Yes
23   img6.jpg  img10.jpg      Yes
24   img6.jpg  img11.jpg      Yes
25   img7.jpg  img10.jpg      Yes
26   img7.jpg  img11.jpg      Yes
27  img10.jpg  img11.jpg      Yes
28   img8.jpg   img9.jpg      Yes
29   img3.jpg  img12.jpg      Yes
30  img13.jpg  img14.jpg      Yes
31  img13.jpg  img15.jpg      Yes
32  img14.jpg  img15.jpg      Yes
33  img16.jpg  img17.jpg      Yes
34  img18.jpg  img19.jpg      Yes
35  img20.jpg  img21.jpg      Yes
36  img22.jpg  img23.jpg      Yes
37  img24.jpg  img25.jpg      Yes

# Negative samples
Compare photos of different people

In [5]:
samples_list = list(idendities.values())

In [6]:
negatives = []

for i in range(0, len(idendities) - 1):
    for j in range(i+1, len(idendities)):
        print(samples_list[i], " vs ",samples_list[j]) 
        cross_product = itertools.product(samples_list[i], samples_list[j])
        cross_product = list(cross_product)
        print(cross_product)
        
        for cross_sample in cross_product:
            print(cross_sample[0], " vs ", cross_sample[1])
            negative = []
            negative.append(cross_sample[0])
            negative.append(cross_sample[1])
            negatives.append(negative)
        

['img1.jpg', 'img2.jpg', 'img4.jpg', 'img5.jpg', 'img6.jpg', 'img7.jpg', 'img10.jpg', 'img11.jpg']  vs  ['img8.jpg', 'img9.jpg']
[('img1.jpg', 'img8.jpg'), ('img1.jpg', 'img9.jpg'), ('img2.jpg', 'img8.jpg'), ('img2.jpg', 'img9.jpg'), ('img4.jpg', 'img8.jpg'), ('img4.jpg', 'img9.jpg'), ('img5.jpg', 'img8.jpg'), ('img5.jpg', 'img9.jpg'), ('img6.jpg', 'img8.jpg'), ('img6.jpg', 'img9.jpg'), ('img7.jpg', 'img8.jpg'), ('img7.jpg', 'img9.jpg'), ('img10.jpg', 'img8.jpg'), ('img10.jpg', 'img9.jpg'), ('img11.jpg', 'img8.jpg'), ('img11.jpg', 'img9.jpg')]
img1.jpg  vs  img8.jpg
img1.jpg  vs  img9.jpg
img2.jpg  vs  img8.jpg
img2.jpg  vs  img9.jpg
img4.jpg  vs  img8.jpg
img4.jpg  vs  img9.jpg
img5.jpg  vs  img8.jpg
img5.jpg  vs  img9.jpg
img6.jpg  vs  img8.jpg
img6.jpg  vs  img9.jpg
img7.jpg  vs  img8.jpg
img7.jpg  vs  img9.jpg
img10.jpg  vs  img8.jpg
img10.jpg  vs  img9.jpg
img11.jpg  vs  img8.jpg
img11.jpg  vs  img9.jpg
['img1.jpg', 'img2.jpg', 'img4.jpg', 'img5.jpg', 'img6.jpg', 'img7.jpg', 'img1

In [7]:
negatives = pd.DataFrame(negatives, columns = ["file_x", "file_y"])
negatives["decision"] = "No"
negatives

file_x     file_y decision
0     img1.jpg   img8.jpg       No
1     img1.jpg   img9.jpg       No
2     img2.jpg   img8.jpg       No
3     img2.jpg   img9.jpg       No
4     img4.jpg   img8.jpg       No
..         ...        ...      ...
257  img21.jpg  img25.jpg       No
258  img22.jpg  img24.jpg       No
259  img22.jpg  img25.jpg       No
260  img23.jpg  img24.jpg       No
261  img23.jpg  img25.jpg       No

[262 rows x 3 columns]

# Merge Positives and Negative Samples

In [8]:
df = pd.concat([positives, negatives]).reset_index(drop = True)

In [9]:
df.shape

(300, 3)

In [10]:
df.decision.value_counts()

No     262
Yes     38
Name: decision, dtype: int64

In [11]:
df

file_x     file_y decision
0     img1.jpg   img2.jpg      Yes
1     img1.jpg   img4.jpg      Yes
2     img1.jpg   img5.jpg      Yes
3     img1.jpg   img6.jpg      Yes
4     img1.jpg   img7.jpg      Yes
..         ...        ...      ...
295  img21.jpg  img25.jpg       No
296  img22.jpg  img24.jpg       No
297  img22.jpg  img25.jpg       No
298  img23.jpg  img24.jpg       No
299  img23.jpg  img25.jpg       No

[300 rows x 3 columns]

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [14]:
# df.file_x = "deepface/tests/dataset/"+df.file_x
# df.file_y = "deepface/tests/dataset/"+df.file_y

df.file_x = "./dataset/"+df.file_x
df.file_y = "./dataset/"+df.file_y

# DeepFace

In [15]:
pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.0 MB/s 
     |████████████████████████████████| 87 kB 5.8 MB/s 
     |████████████████████████████████| 2.3 MB 51.7 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=66b7ef4b2a20f78b611065501ce4eff2d1da492e7be50f41da2de1d935f52a0d
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [16]:
from deepface import DeepFace

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [17]:
instances = df[["file_x", "file_y"]].values.tolist()
instances

[['./dataset/img1.jpg', './dataset/img2.jpg'],
 ['./dataset/img1.jpg', './dataset/img4.jpg'],
 ['./dataset/img1.jpg', './dataset/img5.jpg'],
 ['./dataset/img1.jpg', './dataset/img6.jpg'],
 ['./dataset/img1.jpg', './dataset/img7.jpg'],
 ['./dataset/img1.jpg', './dataset/img10.jpg'],
 ['./dataset/img1.jpg', './dataset/img11.jpg'],
 ['./dataset/img2.jpg', './dataset/img4.jpg'],
 ['./dataset/img2.jpg', './dataset/img5.jpg'],
 ['./dataset/img2.jpg', './dataset/img6.jpg'],
 ['./dataset/img2.jpg', './dataset/img7.jpg'],
 ['./dataset/img2.jpg', './dataset/img10.jpg'],
 ['./dataset/img2.jpg', './dataset/img11.jpg'],
 ['./dataset/img4.jpg', './dataset/img5.jpg'],
 ['./dataset/img4.jpg', './dataset/img6.jpg'],
 ['./dataset/img4.jpg', './dataset/img7.jpg'],
 ['./dataset/img4.jpg', './dataset/img10.jpg'],
 ['./dataset/img4.jpg', './dataset/img11.jpg'],
 ['./dataset/img5.jpg', './dataset/img6.jpg'],
 ['./dataset/img5.jpg', './dataset/img7.jpg'],
 ['./dataset/img5.jpg', './dataset/img10.jpg'],
 ['./d

In [18]:
model_name = "VGG-Face"
distance_metric = "cosine"

In [23]:
# from deepface import DeepFace
# import matplotlib.pyplot as plt
# import glob
# import sys
# import cv2

# models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
# metrics = ["cosine", "euclidean", "euclidean_l2"]

# #1. DB 이미지 리스트 불러오기
# imgs = glob.glob('dataset/*.jpg')

# #2. 이미지 없을 때 예외처리
# if not imgs:
#     print("이미지가 없습니다.")
#     sys.exit()

# #3. DB 이미지 리스트 전처리
# for img in imgs:
#     aaa = cv2.imread(img)
#     plt.imshow(aaa)
#     plt.show()
#     aligned_img = DeepFace.detectFace(img)
#     # plt.imsave(img,aligned_img)

# #4. 입력 이미지 전처리
# # input_img = "img1.jpg"
# # aligned_face = DeepFace.detectFace(input_img)
# # plt.imshow(aligned_face)
# # plt.show()
# # plt.imsave(input_img, aligned_face)

# #5. 모든 모델에 대해 시행

# # for i in range(len(models)):
# #     df = DeepFace.find(img_path=input_img, db_path="./test", model_name=models[i], distance_metric=metrics[2])
# #     print(df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
resp_obj = DeepFace.verify(instances, model_name = model_name, distance_metric = distance_metric)
resp_obj

In [ ]:
distances = []
for i in range(0, len(instances)):
    distance = round(resp_obj["pair_%s" % (i+1)]["distance"], 4)
    distances.append(distance)

In [ ]:
df["distance"] = distances

# Analyzing Distances

In [ ]:
tp_mean = round(df[df.decision == "Yes"].mean().values[0], 4)
tp_std = round(df[df.decision == "Yes"].std().values[0], 4)
fp_mean = round(df[df.decision == "No"].mean().values[0], 4)
fp_std = round(df[df.decision == "No"].std().values[0], 4)

In [ ]:
print("Mean of true positives: ", tp_mean)
print("Std of true positives: ", tp_std)
print("Mean of false positives: ", fp_mean)
print("Std of false positives: ", fp_std)

# Distribution

In [ ]:
df[df.decision == "Yes"].distance.plot.kde()
df[df.decision == "No"].distance.plot.kde()

# Best Split Point

In [ ]:
pip install chefboost

In [ ]:
from chefboost import Chefboost as chef

In [ ]:
config = {'algorithm': 'C4.5'}

In [ ]:
tmp_df = df[['distance', 'decision']].rename(columns = {"decision": "Decision"}).copy()
model = chef.fit(tmp_df, config)

# Sigma

In [ ]:
sigma = 2
#2 sigma corresponds 95.45% confidence, and 3 sigma corresponds 99.73% confidence

#threshold = round(tp_mean + sigma * tp_std, 4)
threshold = 0.3147 #comes from c4.5 algorithm
print("threshold: ", threshold)

In [ ]:
df[df.decision == 'Yes'].distance.max()

In [ ]:
df[df.decision == 'No'].distance.min()

# Evaluation

In [ ]:
df["prediction"] = "No"

In [ ]:
idx = df[df.distance <= threshold].index
df.loc[idx, 'prediction'] = 'Yes'

In [ ]:
df.sample(5)

In [ ]:
cm = confusion_matrix(df.decision.values, df.prediction.values)

In [ ]:
cm

In [ ]:
tn, fp, fn, tp = cm.ravel()

In [ ]:
tn, fp, fn, tp

In [ ]:
recall = tp / (tp + fn)
precision = tp / (tp + fp)
accuracy = (tp + tn)/(tn + fp +  fn + tp)
f1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
print("Precision: ", 100*precision,"%")
print("Recall: ", 100*recall,"%")
print("F1 score ",100*f1, "%")
print("Accuracy: ", 100*accuracy,"%")

In [ ]:
df.to_csv("threshold_pivot.csv", index = False)

## Test results

### Threshold = 0.3147 (C4.5 best split point)

Precision:  100.0 %

Recall:  89.47368421052632 %

F1 score  94.44444444444444%

Accuracy:  98.66666666666667 %

### Threshold = 0.3751 (2 sigma)

Precision:  90.47619047619048 %

Recall:  100.0 %

F1 score  95.0 %

Accuracy:  98.66666666666667 %